In [4]:
import pandas as pd
import numpy as np
import re

# 1. txt 파일에서 URL 추출

In [5]:
# 파일에서 내용 읽어오기
with open("./자청챌린지(9.29~10.6).txt", "r", encoding="utf-8") as file:
    content = file.read()

# 정규 표현식을 사용하여 URL 추출
urls = re.findall(r"https://(?:m\.)?blog\.naver\.com/[\w\-/]+", content)

urls

['https://m.blog.naver.com/hyejin4557/223224430046',
 'https://m.blog.naver.com/fish1fish2/223224430164',
 'https://blog.naver.com/yoo1104/223224431812',
 'https://m.blog.naver.com/aldus5462/223224288871',
 'https://m.blog.naver.com/ljhye919/223224432635',
 'https://blog.naver.com/sonsh86/223224433912',
 'https://m.blog.naver.com/blueplan_/223224433578',
 'https://m.blog.naver.com/thekogwanghwi/223224435405',
 'https://m.blog.naver.com/kye8808/223224436858',
 'https://blog.naver.com/brightyoung00/223224437058',
 'https://blog.naver.com/8651dew/223224437074',
 'https://m.blog.naver.com/bbddlov/223224436527',
 'https://m.blog.naver.com/gamtanfv/223224437560',
 'https://blog.naver.com/qkrrjsan2/223224440182',
 'https://m.blog.naver.com/rlatndms5508/223224442797',
 'https://m.blog.naver.com/mentalisia/223224363221',
 'https://m.blog.naver.com/kjalg1/223224335616',
 'https://m.blog.naver.com/abok_2/223224445097',
 'https://m.blog.naver.com/bocgil_gim/223224448219',
 'https://m.blog.naver.co

# 블로그 아이디당 링크 갯수

In [6]:
# 블로그 아이디 추출
blog_ids = [re.search(r"https://(?:m\.)?blog\.naver\.com/(\w+)", url).group(1) for url in urls if re.search(r"https://(?:m\.)?blog\.naver\.com/(\w+)", url)]

# 아이디별 등장 횟수 카운트
blog_id_counts = {}
for blog_id in blog_ids:
    blog_id_counts[blog_id] = blog_id_counts.get(blog_id, 0) + 1

blog_id_counts_sorted = sorted(blog_id_counts.items(), key=lambda x: x[1], reverse=True)

blog_id_counts_sorted[:10] # 상위 10개만 출력합니다.

[('ywahn1217', 59),
 ('PostView', 31),
 ('yomtong', 19),
 ('yso0716', 14),
 ('syncthink730', 12),
 ('20s_startup', 12),
 ('8651dew', 11),
 ('alswl010105_', 10),
 ('ericwo', 10),
 ('09gsumin', 10)]

In [8]:
from collections import Counter
blog_id_counts = Counter(blog_ids)
blog_id_counts_sorted = sorted(blog_id_counts.items(), key=lambda x: x[1], reverse=True)
blog_id_counts_sorted[:10]

[('ywahn1217', 59),
 ('PostView', 31),
 ('yomtong', 19),
 ('yso0716', 14),
 ('syncthink730', 12),
 ('20s_startup', 12),
 ('8651dew', 11),
 ('alswl010105_', 10),
 ('ericwo', 10),
 ('09gsumin', 10)]

### 중복된 글 링크 제외
- 같은 글을 올린 경우에는 카운팅 제외함. 고유한 글을 올린 경우에만 아이디 카운팅.

In [11]:
# 아이디와 포스팅 ID 함께 추출
blog_id_post_ids = {re.search(r"https://(?:m\.)?blog\.naver\.com/(\w+)/(\d+)", url).groups() for url in urls if re.search(r"https://(?:m\.)?blog\.naver\.com/(\w+)/(\d+)", url)}

# 중복 제거 후 아이디별 등장 횟수 카운트
unique_blog_id_counts = {}
for blog_id, _ in blog_id_post_ids:
    unique_blog_id_counts[blog_id] = unique_blog_id_counts.get(blog_id, 0) + 1

unique_blog_id_counts_sorted = sorted(unique_blog_id_counts.items(), key=lambda x: x[1], reverse=True)

unique_blog_id_counts_sorted[:10]  # 상위 10개만 출력합니다.

[('ywahn1217', 15),
 ('syncthink730', 12),
 ('09gsumin', 10),
 ('changa00207', 9),
 ('bobiekate', 8),
 ('hangdow', 8),
 ('bound_22', 8),
 ('l3ong', 8),
 ('jinyou_p', 8),
 ('minix602', 8)]

In [10]:
blog_id_post_ids = {re.search(r"https://(?:m\.)?blog\.naver\.com/(\w+)/(\d+)", url).groups() for url in urls if re.search(r"https://(?:m\.)?blog\.naver\.com/(\w+)/(\d+)", url)}

unique_blog_id_counts = Counter(blog_id for blog_id, _ in blog_id_post_ids)
unique_blog_id_counts_sorted = sorted(unique_blog_id_counts.items(), key=lambda x: x[1], reverse=True)
unique_blog_id_counts_sorted[:10]

[('ywahn1217', 15),
 ('syncthink730', 12),
 ('09gsumin', 10),
 ('changa00207', 9),
 ('bobiekate', 8),
 ('hangdow', 8),
 ('bound_22', 8),
 ('l3ong', 8),
 ('jinyou_p', 8),
 ('minix602', 8)]

### 매일 글 1개 올린 참여자

In [4]:
# 날짜 구분자로 텍스트 분리
date_separators = re.findall(r"--------------- (\d{4}년 \d{1,2}월 \d{1,2}일 \w요일) ---------------", content)
date_sections = re.split(r"--------------- \d{4}년 \d{1,2}월 \d{1,2}일 \w요일 ---------------", content)[1:]

# 각 날짜 구간에 등장하는 블로그 아이디 추출
date_blog_ids = {}
for date, section in zip(date_separators, date_sections):
    ids = list({re.search(r"https://(?:m\.)?blog\.naver\.com/(\w+)", url).group(1) for url in re.findall(r"https://(?:m\.)?blog\.naver\.com/\w+", section)})
    date_blog_ids[date] = ids

# 9월 29일부터 10월 6일까지 매일 참여한 사람들 찾기
consistent_participants = set(date_blog_ids[date_separators[0]])
for date in date_separators[1:]:
    consistent_participants &= set(date_blog_ids[date])

# consistent_participants에 해당하는 블로그의 기본 URL 생성
consistent_participant_urls = ["https://m.blog.naver.com/" + participant for participant in consistent_participants]
consistent_participant_urls

['https://m.blog.naver.com/l3ong',
 'https://m.blog.naver.com/wksel0305',
 'https://m.blog.naver.com/happylife0211',
 'https://m.blog.naver.com/bobiekate',
 'https://m.blog.naver.com/PostView',
 'https://m.blog.naver.com/09gsumin',
 'https://m.blog.naver.com/kjalg1',
 'https://m.blog.naver.com/vvvivid_life',
 'https://m.blog.naver.com/hak_yoon',
 'https://m.blog.naver.com/leesol1280',
 'https://m.blog.naver.com/lwd0807',
 'https://m.blog.naver.com/blueplan_',
 'https://m.blog.naver.com/minix602',
 'https://m.blog.naver.com/ywahn1217',
 'https://m.blog.naver.com/changa00207',
 'https://m.blog.naver.com/syncthink730',
 'https://m.blog.naver.com/yso0716',
 'https://m.blog.naver.com/jinyou_p']

In [87]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

[(f.name, f.fname) for f in fm.fontManager.ttflist if 'Nanum' in f.name]

[('NanumMyeongjo', 'C:\\Windows\\Fonts\\NanumMyeongjoExtraBold.ttf'),
 ('NanumMyeongjo', 'C:\\Windows\\Fonts\\NanumMyeongjoBold.ttf'),
 ('NanumGothic', 'C:\\Windows\\Fonts\\NanumGothic.ttf'),
 ('NanumGothic', 'C:\\Windows\\Fonts\\NanumGothicExtraBold.ttf'),
 ('NanumMyeongjo', 'C:\\Windows\\Fonts\\NanumMyeongjo.ttf'),
 ('NanumGothic', 'C:\\Windows\\Fonts\\NanumGothicBold.ttf'),
 ('NanumGothic', 'C:\\Windows\\Fonts\\NanumGothicLight.ttf')]

In [29]:
plt.rc('font', family='NanumGothic')
mpl.rcParams['axes.unicode_minus'] = False

### 참여 횟수별 참여자 수

In [11]:
# 각 참여 횟수별 아이디 수 카운트
participation_freq = {}
for count in id_participation_counts.values():
    participation_freq[count] = participation_freq.get(count, 0) + 1

# 참여 횟수별로 아이디 수를 출력
print("참여 횟수별 아이디 수:")
for count, id_count in participation_freq.items():
    print(f"{count}번 참여한 아이디 수: {id_count}명")

NameError: name 'id_participation_counts' is not defined

In [10]:
import matplotlib.pyplot as plt

# 각 참여 횟수별 아이디 수 카운트
participation_freq = {}
for count in id_participation_counts.values():
    participation_freq[count] = participation_freq.get(count, 0) + 1

# 데이터로 그래프 그리기
x = list(participation_freq.keys())
y = list(participation_freq.values())

plt.figure(figsize=(10, 6))
bars = plt.bar(x, y, color='skyblue')
plt.xlabel('참여 횟수')
plt.ylabel('참여자 수')
plt.title('챌린지 참여 횟수별 통계')
plt.xticks(x)
plt.ylim(0, 140)
plt.grid(axis='y')

# 막대 위에 카운트 표시
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 3, int(yval), ha='center', va='bottom')

plt.tight_layout()
plt.show()

NameError: name 'id_participation_counts' is not defined

- 작심 삼일ㅋㅋㅋㅋㅋㅋㅋㅋ

### 날짜별로 참여자 수

In [6]:
# 날짜별 아이디 개수 세기
date_id_counts = {date: len(ids) for date, ids in new_date_blog_ids.items()}

date_id_counts

NameError: name 'new_date_blog_ids' is not defined

In [14]:
# 날짜를 key로, 해당 날짜에 포스팅한 아이디의 set을 value로 가지는 딕셔너리를 생성합니다.
date_blog_ids = {}
for date, blog_id in zip(dates, blog_ids):  # dates와 blog_ids는 이전 단계에서 생성한 리스트입니다.
    if date not in date_blog_ids:
        date_blog_ids[date] = set()
    date_blog_ids[date].add(blog_id)

# 날짜별 아이디 개수를 계산합니다.
date_id_counts = {date: len(ids) for date, ids in date_blog_ids.items()}

# 결과 출력
print("날짜별 아이디 개수:", date_id_counts)


NameError: name 'dates' is not defined

In [9]:
# 라벨 형식 변경
def convert_date_format(date_str):
    year, month, day = re.search(r'(\d{4})년 (\d{1,2})월 (\d{1,2})일', date_str).groups()
    day_of_week = re.search(r'(\w)요일', date_str).group(1)
    return f"{year[2:]}.{month.zfill(2)}.{day.zfill(2)} ({day_of_week})"

# x축 라벨 변환
converted_dates = [convert_date_format(date) for date in dates]

plt.figure(figsize=(12, 6))
plt.bar(converted_dates, counts, color='lightsalmon')
plt.ylabel('참여자 수')
plt.title('챌린지 날짜별 참여 통계')
#plt.xticks(rotation=30)
plt.ylim(100, 350)
plt.grid(axis='y')

plt.tight_layout()
plt.show()

NameError: name 'dates' is not defined

In [8]:
# "1일차", "2일차", ... 로 변경된 x축 라벨 생성
day_labels = [f"{i}일차" for i in range(1, len(dates) + 1)]

plt.figure(figsize=(12, 6))
bars = plt.bar(day_labels, counts, color='lightsalmon')
plt.ylabel('참여자 수')
plt.title('챌린지 일자별 참여 통계')
plt.ylim(100, 350)
plt.grid(axis='y')

# 막대 위에 카운트 표시
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 5, int(yval), ha='center', va='bottom')

plt.tight_layout()
plt.show()

NameError: name 'dates' is not defined

- 9월 30일(토) 내일부터 하자!
- 10월 2일 (월) 월요일인데 한번 해볼까?
- 10월 4일 (수) 출근했더니 피곤하다..ㅎ
- ...
- 308명에서 209명이면 나쁘지 않은 수치다. 

### 한번이라도 참여한 사람


In [1]:
# 전체 기간 동안 등장한 고유한 아이디 추출
unique_ids_overall = set()
for ids in new_date_blog_ids.values():
    unique_ids_overall.update(ids)

# 고유한 아이디 개수 계산
unique_id_count = len(unique_ids_overall)
unique_id_count # 532개

NameError: name 'new_date_blog_ids' is not defined

In [13]:
unique_ids_overall = set(blog_ids)
unique_id_count = len(unique_ids_overall)
unique_id_count

532

## 이제 서로이웃추가하러 가자

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def add_friends_and_log(id_recent_urls):
    # 웹드라이버 초기화
    service = Service('C:/chromedriver-win64/chromedriver.exe')
    driver = webdriver.Chrome(service=service)

    if not os.path.exists('visited_urls.txt'):
        with open('visited_urls.txt', 'w') as f:
            pass

    with open('visited_urls.txt', 'r') as f:
        visited_urls = set(f.readlines())

    added_today = 0
    try:
        for url in id_recent_urls:
            # URL을 m.blog.naver.com 형식으로 변경
            mobile_url = url.replace('blog.naver.com', 'm.blog.naver.com')
            
            if mobile_url in visited_urls:
                continue
            driver.get(mobile_url)
            time.sleep(3)

            try:
                # 이웃추가 버튼을 찾아 클릭
                add_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//a[contains(text(), '이웃추가')]"))
                )
                add_button.click()
            except:
                # 이웃추가 버튼이 없으면 (이미 서로이웃 추가한 경우) 다음 URL로 넘어감
                continue

            with open('visited_urls.txt', 'a') as f:
                f.write(mobile_url + '\n')
            visited_urls.add(mobile_url)
            added_today += 1
    finally:
        driver.quit()

    current_date = datetime.now().strftime('%Y-%m-%d')
    with open('addition_log.txt', 'a') as log_file:
        log_file.write(f"{current_date}: {added_today}명에게 이웃추가\n")

# 코드 실행
participation_counts = [8, 7, 6, 5, 4, 3, 2, 1]
id_recent_urls = list_up_ids_by_participation(participation_counts, id_participation_counts, urls)
add_friends_and_log(id_recent_urls)